In [1]:
import pandas as pd
import gzip
import os
import numpy as np

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
arch_sellout = "datasets/emp3_sellout_base.csv"
arch_cluster_prefijo = "datasets/emp3_sellout_base_cluster_"

In [4]:
df_sellout = pd.read_csv(arch_sellout)
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19639107 entries, 0 to 19639106
Data columns (total 26 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   product_id                 int64  
 1   periodo                    int64  
 2   customer_id                int64  
 3   cero_ventas                int64  
 4   plan_precios_cuidados      float64
 5   cust_request_qty           float64
 6   cust_request_tn            float64
 7   tn                         float64
 8   periodo_fecha              object 
 9   mes                        int64  
 10  meses_historia_prod        int64  
 11  meses_historia_cliente     int64  
 12  cliente_estrella           int64  
 13  prod_cust                  object 
 14  cat1                       object 
 15  cat2                       object 
 16  cat3                       object 
 17  brand                      object 
 18  sku_size                   float64
 19  producto_estrella          float64
 20  

In [5]:
df_sellout.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,...,cat3,brand,sku_size,producto_estrella,temp_media,temp_max_media,temp_min_media,IPC,promedio_mens_dolar_venta,catastrofe
0,20001,201701,10001,0,0.0,11.0,99.43861,99.43861,2017-01-01,1,...,Liquido,ARIEL,3000.0,1.0,25.65,30.4,20.9,1.6,15.91,False
1,20001,201702,10001,0,0.0,23.0,198.84365,198.84365,2017-02-01,2,...,Liquido,ARIEL,3000.0,1.0,25.35,30.1,20.6,2.1,15.59,False
2,20001,201703,10001,0,0.0,33.0,92.46537,92.46537,2017-03-01,3,...,Liquido,ARIEL,3000.0,1.0,22.45,26.9,18.0,2.4,15.52,False
3,20001,201704,10001,0,0.0,8.0,13.29728,13.29728,2017-04-01,4,...,Liquido,ARIEL,3000.0,1.0,18.90,23.5,14.3,2.7,15.36,False
4,20001,201705,10001,0,0.0,15.0,101.20711,101.00563,2017-05-01,5,...,Liquido,ARIEL,3000.0,1.0,15.90,19.5,12.3,1.4,15.72,False


## Clustering Manual

In [6]:
# Armo primer cluster con productos y clientes estrella
df_sellout_cluster_estrella = df_sellout[(df_sellout.cliente_estrella==1)&(df_sellout.producto_estrella==1)]
print("Cluster Estrella:",len(df_sellout_cluster_estrella))
df_sellout_no_estrella = df_sellout[~((df_sellout.cliente_estrella==1)&(df_sellout.producto_estrella==1))]
print("Resto:",len(df_sellout_no_estrella))

Cluster Estrella: 4020
Resto: 19635087


In [7]:
df_sellout_cluster_estrella.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,...,cat3,brand,sku_size,producto_estrella,temp_media,temp_max_media,temp_min_media,IPC,promedio_mens_dolar_venta,catastrofe
0,20001,201701,10001,0,0.0,11.0,99.43861,99.43861,2017-01-01,1,...,Liquido,ARIEL,3000.0,1.0,25.65,30.4,20.9,1.6,15.91,False
1,20001,201702,10001,0,0.0,23.0,198.84365,198.84365,2017-02-01,2,...,Liquido,ARIEL,3000.0,1.0,25.35,30.1,20.6,2.1,15.59,False
2,20001,201703,10001,0,0.0,33.0,92.46537,92.46537,2017-03-01,3,...,Liquido,ARIEL,3000.0,1.0,22.45,26.9,18.0,2.4,15.52,False
3,20001,201704,10001,0,0.0,8.0,13.29728,13.29728,2017-04-01,4,...,Liquido,ARIEL,3000.0,1.0,18.90,23.5,14.3,2.7,15.36,False
4,20001,201705,10001,0,0.0,15.0,101.20711,101.00563,2017-05-01,5,...,Liquido,ARIEL,3000.0,1.0,15.90,19.5,12.3,1.4,15.72,False


In [8]:
df_sellout_cluster_estrella.to_csv(arch_cluster_prefijo + "0.csv",index=False)

In [9]:
pares_prod_cli = df_sellout_no_estrella[["prod_cust"]].drop_duplicates()
print(len(pares_prod_cli))
pares_prod_cli.head()

735981


,prod_cust
324,20001-10010
468,20001-10014
504,20001-10015
540,20001-10016
576,20001-10017


In [10]:
pares_prod_cli = pares_prod_cli.sample(5000)
len(pares_prod_cli)

5000

In [11]:
cluster_count = 1
cluster_size = int(np.ceil(len(pares_prod_cli)/cluster_count))
print("Total:",len(pares_prod_cli))

for cluster_iter in range(1,cluster_count+1):
    #Tomo de muestra el tamaño del cluster o lo que tiene el dataframe
    sample_size = cluster_size
    if sample_size >  len(pares_prod_cli):
        sample_size = len(pares_prod_cli)
    
    print(cluster_iter,"-", sample_size)

    
    cluster = pares_prod_cli.sample(sample_size)
    pares_prod_cli = pares_prod_cli.drop(cluster.index)
    df_cluster = df_sellout_no_estrella[(df_sellout_no_estrella.prod_cust.isin(cluster.prod_cust))]
    df_cluster.to_csv(arch_cluster_prefijo + str(cluster_iter) + ".csv",index=False)

Total: 5000
1 - 5000


In [12]:
df_cluster.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133430 entries, 9966 to 19638160
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   product_id                 133430 non-null  int64  
 1   periodo                    133430 non-null  int64  
 2   customer_id                133430 non-null  int64  
 3   cero_ventas                133430 non-null  int64  
 4   plan_precios_cuidados      133430 non-null  float64
 5   cust_request_qty           133430 non-null  float64
 6   cust_request_tn            133430 non-null  float64
 7   tn                         133430 non-null  float64
 8   periodo_fecha              133430 non-null  object 
 9   mes                        133430 non-null  int64  
 10  meses_historia_prod        133430 non-null  int64  
 11  meses_historia_cliente     133430 non-null  int64  
 12  cliente_estrella           133430 non-null  int64  
 13  prod_cust               

In [13]:
df_cluster.head()

,product_id,periodo,customer_id,cero_ventas,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,periodo_fecha,mes,...,cat3,brand,sku_size,producto_estrella,temp_media,temp_max_media,temp_min_media,IPC,promedio_mens_dolar_venta,catastrofe
9966,20001,201909,10283,1,0.0,0.0,0.00000,0.00000,2019-09-01,9,...,Liquido,ARIEL,3000.0,1.0,15.30,20.4,10.4,5.9,56.46,False
9967,20001,201910,10283,1,0.0,0.0,0.00000,0.00000,2019-10-01,10,...,Liquido,ARIEL,3000.0,1.0,17.80,22.0,13.0,3.3,58.54,False
9968,20001,201911,10283,0,0.0,2.0,0.11202,0.11202,2019-11-01,11,...,Liquido,ARIEL,3000.0,1.0,22.90,27.8,17.6,4.3,59.75,False
9969,20001,201912,10283,1,0.0,0.0,0.00000,0.00000,2019-12-01,12,...,Liquido,ARIEL,3000.0,1.0,24.00,29.5,18.0,3.7,59.88,False
11841,20001,201701,10339,1,0.0,0.0,0.00000,0.00000,2017-01-01,1,...,Liquido,ARIEL,3000.0,1.0,25.65,30.4,20.9,1.6,15.91,False
